# User guide on the StructureData class

The atomistic `StructureData` class is basically an enhanced version of the `orm.StructureData`, which was implemented in `aiida-core`. 
Relevante changes, which  are:
- `properties` attribute, used to store all the properties associated the the crystal structure;
- the kind-based definition of the structure is *no more supported*, in favour of a code-agnostic site-based definition of the properties;
- the StructureData node is now really a *data container*, meaning that we do not have methods to modify it after its creation, i.e. it is *immutable* even before the store of the node; 

<div style="border:2px solid #f7d117; padding: 10px; margin: 10px 0;">
    <strong>Why we support only site-based definition of properties:</strong> This simplifies multiple properties defintion and respect the philosophy of a code-agnostic representation of the structure. The kinds determination can be done using the built-in `get_kinds()` method of the StructureData. It is also possible to provide a user-defined set of kinds via *tags*.
</div>

<div style="border:2px solid #f7d117; padding: 10px; margin: 10px 0;">
    <strong>Backward compatibility:</strong> .
</div>

## Properties
Properties as divided in three main domains:  *global*, *intra-site*, and *inter-site*. 
Examples are:

global:
  - cell
  - periodic boundary conditions (PBC)

intra-site:
  - positions
  - symbols 
  - masses
  - electronic charge
  - magnetization - TOBE added
  - Hubbard U parameters - TOBE added

inter-site:
  - Hubbard V parameters - TOBE added 

Some of these properties are related to the sites/atoms (e.g. atomic positions, symbols, electronic charge) and some are related to the whole structure (e.g. PBC, cell). So, each property will have an attribute `domain`, which can be "intra-site", "inter-site", "global". 

## Custom properties
The possibility to have user defined custom properties is discussed in another section (TOBE ADDED).

## Available properties
To explore the available properties in detail, please go to the corresponding pages (TOBE ADDED).

In [15]:
from aiida import orm, load_profile
load_profile()

from aiida_atomistic.data.structure import StructureData

## The first StructureData instance
One of the principle of the new StructureData is the fact that it is "just" a container of the information about a given structure: this means that, after that instances of this class are immutable. After the initialization, it is not possible to change the stored properties.

Properties should be contained in a dictionary:

In [28]:
properties_dict = {
    "cell":{"value":[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]},
    "positions":{"value":[[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]],},
    "symbols":{"value":["Li","Li"]},
    }

where the value of each defined property is defined under the corresponding dictionary, as value of the key `value`. 


In [41]:
print(f"The whole list of currently supported properties is: \n{StructureData().properties.get_supported_properties()}")

The whole list of currently supported properties is: 
['cell', 'pbc', 'custom', 'mass', 'positions', 'charge', 'symbols']


To initialise a StructureData node is then sufficient to do:

In [47]:
structure = StructureData(properties = properties_dict)
structure

<StructureData: uuid: 13af7274-5a68-4f82-8dd3-32a5b616ae0f (unstored)>

we can inspect the properties by accessing the corresponding attribute (tab completion is enabled):

In [48]:
print(f"The cell property class: \n{structure.properties.cell}\n")
print(f"The cell property value: \n{structure.properties.cell.value}\n")
print(f"The cell property domain: \n{structure.properties.cell.domain}\n")

The cell property class: 
parent=<StructureData: uuid: 13af7274-5a68-4f82-8dd3-32a5b616ae0f (unstored)> value=[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]] domain='global'

The cell property value: 
[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]

The cell property domain: 
global



In [49]:
print(f"The positions property class: \n{structure.properties.positions}\n")
print(f"The positions property value: \n{structure.properties.positions.value}\n")
print(f"The positions property domain: \n{structure.properties.positions.domain}\n")

The positions property class: 
parent=<StructureData: uuid: 13af7274-5a68-4f82-8dd3-32a5b616ae0f (unstored)> domain='intra-site' value=[[0.0, 0.0, 0.0], [1.5, 1.5, 1.5]] kind_tags=None

The positions property value: 
[[0.0, 0.0, 0.0], [1.5, 1.5, 1.5]]

The positions property domain: 
intra-site



In [50]:
print(f"Stored properties are: \n{structure.properties.get_stored_properties()}")

Stored properties are: 
['cell', 'positions', 'symbols']


# HERE

Additionally, we can access the property using the `get_property_attribute` method:

In [51]:
structure.properties.get_property_attribute("mass")

KeyError: 'mass'

### <a id='toc1_1_2_'></a>[Inspect the supported and stored properties from the StructureData instance:](#toc0_)

In [23]:
structure.properties.get_supported_properties()

['pbc', 'mass', 'custom', 'cell', 'symbols', 'charge', 'positions']

In [24]:
structure.properties.get_stored_properties()

['mass', 'cell', 'pbc', 'symbols', 'charge', 'positions']

#### <a id='toc1_1_2_1_'></a>[Missing: access the supported property from the class object](#toc0_)

```python
In [1]: StructureData.get_supported_properties()
Out [2]: ['custom', 'pbc', 'symbols', 'charge', 'positions', 'mass', 'cell']
```

### <a id='toc1_1_3_'></a>[The immutability of the StructureData instance](#toc0_)

A crucial aspect of the new `StructureData` is that it is immutable even if the node is not stored, i.e. the API does not support on-the-fly or interactive modifications (it will raise errors). This helps in avoiding unexpected 
behaviour coming from a step-by-step defintion of the structure, e.g. incosistencies between properties definitions, which are then not cross-checked again.

One has to define a new `StructureData` instance by scratch.
To make user life simpler, we provide a `to_dict` method, which can be used to generate the properties dictionary. This can be updated and used for a new StructureData instance:

In [25]:
new_properties = structure.to_dict()
new_properties

{'cell': {'value': [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]},
 'pbc': {'value': [True, True, True]},
 'positions': {'value': [[0.0, 0.0, 0.0], [1.5, 1.5, 1.5]]},
 'symbols': {'value': ['Li', 'Li']},
 'mass': {'value': [6.941, 6.945]},
 'charge': {'value': [1, 0]}}

In [26]:
new_properties["mass"]["value"] = [6.941,6.941]

In [27]:
new_structure = StructureData(properties=new_properties)

In [28]:
new_structure.properties.mass.value

[6.941, 6.941]

### <a id='toc1_1_4_'></a>[How to get kinds](#toc0_)

It is possible to get a list of kinds using the `get_kinds` method. This will generate the corresponding predicted kinds for all the properties (the "intra-site" ones) and then generate the list of global different kinds. 


In [29]:
structure.get_kinds()

Li
Li


([0, 1],
 ['Li0', 'Li1'],
 {'mass': [6.9415, 6.945500000000001], 'charge': [1.1250000000000002, 0.225]})

As you can see, basically the value of a property is not the starting value choosen, and this is a PROBLEM: we have TOFIX this, especially in case in which the value is zero. 
The issue is that we are using a middle-point representative value.

Kinds are determined using, for each property, a given threshold. There is a default threshold:

In [31]:
structure.properties.mass.default_kind_threshold

0.001

Basically, it will be possible to override the threshold to be used for each property (TOBE implemented):

```python
structure.get_kinds(thr={"mass":0.1})
```

#### User defined kinds

it is possible to define kinds by hands, when the StructureData instance is generated.

```python
properties = {
    ...
    "positions":{
        "value":[...],
        "kind_tags":["Li0","Li0"],
    }
    ...
}
```
So that a given plugin will use the given tags, if checks that `structure.properties.positions.kind_tags`exists.

"